# Openlimits Python Example

In [2]:
from openlimits_python import ExchangeClient
from dotenv import load_dotenv
import os

## Setup

Initialize a new client from API key data. Exchanges take different initialization params

In [4]:
# this allows python to pull ENV variables from a .env file, nothing to do with the API
load_dotenv()

# exchange initialization and credentials for nash
nash_creds = {
    "nash": {
        "credentials": {
            "nash_credentials": {
                "secret": os.getenv("NASH_API_SECRET"),
                "session": os.getenv("NASH_API_KEY")
            },
        },
        "client_id": 0,
        "environment": "production",
        "timeout": 10000
    }
}

# other exchanges take slightly different initialization credentials
binance_creds = {
    "binance": {
        "credentials": None, # we could provide API keys here
        "sandbox": False,
    }
}

In [5]:
# initialize a Nash client
client = ExchangeClient(nash_creds)

## Account and Market queries

You can find the full set of queries supported [here](https://github.com/nash-io/openlimits-python/blob/master/src/lib.rs)

In [6]:
client.get_account_balances(None)[:5] # None is for pagination + just sample 5

[{'balance': {'asset': 'gunthy',
   'free': '5.5814223834375000000000000',
   'total': '5.5814223834375000000000000'}},
 {'balance': {'asset': 'qnt',
   'free': '0.0011742091125000000000000',
   'total': '0.0011742091125000000000000'}},
 {'balance': {'asset': 'ant',
   'free': '0.0006148462500000000000000',
   'total': '0.0006148462500000000000000'}},
 {'balance': {'asset': 'zrx',
   'free': '0.3049423050000000000000000',
   'total': '0.3049423050000000000000000'}},
 {'balance': {'asset': 'eth',
   'free': '0.0295369258034560000000000',
   'total': '0.1295369258034560000000000'}}]

In [6]:
client.get_order_history("btc_usdc", None)[:2] # None for pagination + just look at first two

[{'order': {'id': '1603597218370000000',
   'market_pair': 'btc_usdc',
   'price': '13287.60000000',
   'order_type': {'order_type': 'limit'},
   'client_order_id': None,
   'created_at': 1603597218367,
   'side': {'side': 'sell'},
   'size': '0.000383810',
   'status': {'order_status': 'filled'}}},
 {'order': {'id': '1603557123448000000',
   'market_pair': 'btc_usdc',
   'price': '13074.40000000',
   'order_type': {'order_type': 'limit'},
   'client_order_id': None,
   'created_at': 1603557123446,
   'side': {'side': 'buy'},
   'size': '0.000986660',
   'status': {'order_status': 'filled'}}}]

In [9]:
client.limit_buy("eth_usdc", "0.0171", "350.46")

{'order': {'id': '1603921615278000000',
  'market_pair': 'eth_usdc',
  'price': None,
  'order_type': {'order_type': 'limit'},
  'client_order_id': None,
  'created_at': 1603921615274,
  'side': {'side': 'buy'},
  'size': '0',
  'status': {'order_status': 'pending'}}}

In [10]:
client.get_order("1603921615278000000")

{'order': {'id': '1603921615278000000',
  'market_pair': 'eth_usdc',
  'price': '350.46000000',
  'order_type': {'order_type': 'limit'},
  'client_order_id': None,
  'created_at': 1603921615274,
  'side': {'side': 'buy'},
  'size': '0.017100',
  'status': {'order_status': 'open'}}}

In [11]:
client.cancel_order("eth_usdc", "1603921615278000000")

{'order_canceled': '1603921615278000000'}

In [7]:
client.get_historic_trades("btc_usdc", None)[:2] # None is for pagination + just look at first two

[{'trade': {'liquidity': {'liquidity': 'maker'},
   'market_pair': 'btc_usdc',
   'price': '13636.50000000',
   'qty': '0.000652850',
   'order_id': '1603825447003000000',
   'side': {'side': 'buy'},
   'created_at': 1603826518114,
   'fees': '0',
   'id': 'NWODxQWUdCn34pa8Vydcz-84vjKefwH1jQ46opPkSAE'}},
 {'trade': {'liquidity': {'liquidity': 'maker'},
   'market_pair': 'btc_usdc',
   'price': '13636.50000000',
   'qty': '0.001467000',
   'order_id': '1603825446980000001',
   'side': {'side': 'buy'},
   'created_at': 1603826518114,
   'fees': '0',
   'id': 'Tt25Xq82l-sgME_kW5Y3kW_mp1XvVhCeKLHHIPivH_8'}}]

## Subscriptions

Subscriptions work by registering a callback function with the client. This callback function will be passed every incoming event on the subscription. The callback executes within a seperate thread managed by the openlimits Rust library. The client instance and other state may be accessed within the callback via declared `global` variables.

In [32]:
state = {"num_events": 0, "events":[]} # a simple example of state

# this will be our callback function
def process_event(event):
    global client # we can use this to access the client and react to events
    global state
    
    state["num_events"] += 1
    state["events"].append(event)

In [33]:
# now we register the callback with the client
client.set_subscription_callback(process_event)

In [34]:
# now we subscribe to the orderbook
# the second param limits the number of events we get back. we set to 0 as it is not used at the moment
client.subscribe({"orderbook":("btc_usdc", 0)}) 

In [35]:
# now we wait a few moments and inspect the state
print(state["num_events"])

2


In [36]:
# here is the latest incoming event
print(state["events"][-1])

{'orderbook': {'asks': [{'ask_or_bid': {'price': '13242.60000000', 'qty': '0.09278807'}}, {'ask_or_bid': {'price': '13252.40000000', 'qty': '0.00264103'}}, {'ask_or_bid': {'price': '13253.10000000', 'qty': '0.00000000'}}, {'ask_or_bid': {'price': '13253.20000000', 'qty': '3.51262434'}}], 'bids': []}}


## Multiple clients

You can run multiple openlimits clients at the same time. The underlying Rust library will spawn multiple processes with independent runtimes. For example, we can initialize a Binance client simultaneously:

In [8]:
binance_client = ExchangeClient(binance_creds)

We can use it to count the number of available markets on Binance

In [13]:
print(len(binance_client.list_markets()))
print(binance_client.list_markets()[-1])

1137
{'market_pair': {'quote': 'ETH', 'quote_decimal': '0.00000001', 'base': 'POWR', 'base_increment': '1.00000000', 'symbol': 'POWRETH'}}


Notably, we can use the same API for callbacks here as well. This will only process events from Binance

In [15]:
binance_events = []

def callback_for_binance(event):
    global binance_events
    binance_events.append(event)
    
binance_client.set_subscription_callback(callback_for_binance)

In [30]:
# note that market names still differ between exchanges!
binance_client.subscribe({"orderbook": ("btcusdt", 5)})

In [37]:
binance_events[-1]

{'orderbook': {'asks': [{'ask_or_bid': {'price': '13226.23000000',
     'qty': '12.48326600'}},
   {'ask_or_bid': {'price': '13226.39000000', 'qty': '0.00900000'}},
   {'ask_or_bid': {'price': '13226.54000000', 'qty': '0.37060500'}},
   {'ask_or_bid': {'price': '13226.88000000', 'qty': '0.10417300'}},
   {'ask_or_bid': {'price': '13227.03000000', 'qty': '0.00900000'}}],
  'bids': [{'ask_or_bid': {'price': '13226.22000000', 'qty': '0.00900000'}},
   {'ask_or_bid': {'price': '13226.19000000', 'qty': '0.00201700'}},
   {'ask_or_bid': {'price': '13226.01000000', 'qty': '0.18000000'}},
   {'ask_or_bid': {'price': '13226.00000000', 'qty': '0.00100000'}},
   {'ask_or_bid': {'price': '13225.16000000', 'qty': '0.00078500'}}]}}